# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

import fmputils as fmp

In [2]:
fmp_free_stocks = [
    "AAPL", "TSLA", "AMZN", "MSFT", "NVDA", "GOOGL", "META", "NFLX", "JPM", "V", 
    "BAC", "AMD", "PYPL", "DIS", "T", "PFE", "COST", "INTC", "KO", "TGT", "NKE", 
    "SPY", "BA", "BABA", "XOM", "WMT", "GE", "CSCO", "VZ", "JNJ", "CVX", "PLTR", 
    "SQ", "SHOP", "SBUX", "SOFI", "HOOD", "RBLX", "SNAP", "AMD", "UBER", "FDX", 
    "ABBV", "ETSY", "MRNA", "LMT", "GM", "F", "RIVN", "LCID", "CCL", "DAL", "UAL", 
    "AAL", "TSM", "SONY", "ET", "NOK", "MRO", "COIN", "RIVN", "SIRI", "SOFI", 
    "RIOT", "CPRX", "PYPL", "TGT", "VWO", "SPYG", "NOK", "ROKU", "HOOD", "VIAC", 
    "ATVI", "BIDU", "DOCU", "ZM", "PINS", "TLRY", "WBA", "VIAC", "MGM", "NFLX", 
    "NIO", "C", "GS", "WFC", "ADBE", "PEP", "UNH", "CARR", "FUBO", "HCA", "TWTR", 
    "BILI", "SIRI", "VIAC", "FUBO", "RKT"
]
fmp_free_stocks = list(set(fmp_free_stocks))    # remove duplicates


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.tail()

,Ticker
500,YUM
501,ZBH
502,ZBRA
503,ZION
504,ZTS


In [3]:
# Replace every "." character with "-", because FMP cannot read e.g. "BF.B", but it instead accepts "BF-B"
print(stocks.iloc[61,0])
stocks = stocks.map(lambda x: x.replace('.', '-') if isinstance(x, str) else x)
print(stocks.iloc[61,0])

BF.B
BF-B


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
data = fmp.company_profile(symbol='AAPL')
data

[{'symbol': 'AAPL',
  'price': 220.94,
  'marketCap': 3318982774000,
  'beta': 1.178,
  'lastDividend': 1,
  'range': '164.08-260.1',
  'change': -6.54,
  'changePercentage': -2.87498,
  'volume': 21970967,
  'averageVolume': 52016508,
  'companyName': 'Apple Inc.',
  'currency': 'USD',
  'cik': '0000320193',
  'isin': 'US0378331005',
  'cusip': '037833100',
  'exchangeFullName': 'NASDAQ Global Select',
  'exchange': 'NASDAQ',
  'industry': 'Consumer Electronics',
  'website': 'https://www.apple.com',
  'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that 

In [ ]:
price = data[0]['price']
mk_cap = data[0]['marketCap']/1000000000000
print(price, mk_cap)

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [4]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [ ]:
s = pd.Series(['AAPL', price, mk_cap, 'N/A'], index = my_columns)

final_dataframe = pd.concat([final_dataframe, s.to_frame().T], ignore_index = True)
final_dataframe

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
skipped = []

for symbol in stocks['Ticker']:

    data = fmp.company_profile(symbol=symbol)
    print(data)

    if data:    # if data is not empty (i.e. FMP was able to retrieve information)
        s = pd.Series([symbol, data[0]['price'], data[0]['marketCap'], 'N/A'], index = my_columns)
        final_dataframe = pd.concat([final_dataframe, s.to_frame().T], ignore_index = True)
    else:
        skipped.append(symbol)

In [ ]:
final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [6]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
#symbol_groups

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

symbol_strings[0]
data = fmp.batch_company_profile(symbols=symbol_strings[0])

In [8]:
data
#len(data)
data[3]['price']

219.06

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for batch in symbol_strings:

    data = fmp.batch_company_profile(symbols=batch)     # the batch call already filters out non-retrievable assets
    print(len(data))

    for symbol, stock in zip(batch.split(','), data):
            
        s = pd.Series([symbol, stock['price'], stock['marketCap'], 'N/A'], index = my_columns)
        final_dataframe = pd.concat([final_dataframe, s.to_frame().T], ignore_index = True)

100
95
99
97
98
5


In [8]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,120.015,34216636545,N/A
1,AAL,11.015,7243199640,N/A
2,AAP,35.74,2136998246,N/A
3,AAPL,216.785,3256565948500,N/A
4,ABBV,212.17,374554309500,N/A
...,...,...,...,...
489,YUM,153.165,42748657830,N/A
490,ZBH,104.185,20739378655,N/A
491,ZBRA,281.3275,14454381888,N/A
492,ZION,48.15,7120999800,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we still haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [9]:
while True:
    try:
        portfolio_size = float(input("Enter the value of your portfolio: "))
        break
    except ValueError:
        print("Please input a valid number. Try again.")

print(f'Selected portfolio size: ${portfolio_size} USD')

Selected portfolio size: $1000000.0 USD


In [10]:
# Position Size: how much money are you going to invest in each stock.
position_size = portfolio_size / len(final_dataframe.index)
print(f'Position Size: ${position_size:.2f} USD')


# Calculate how many shares can we buy in each stock with the given Position Size
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe

Position Size: $2024.29 USD


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,120.015,34216636545,16
1,AAL,11.015,7243199640,183
2,AAP,35.74,2136998246,56
3,AAPL,216.785,3256565948500,9
4,ABBV,212.17,374554309500,9
...,...,...,...,...
489,YUM,153.165,42748657830,13
490,ZBH,104.185,20739378655,19
491,ZBRA,281.3275,14454381888,7
492,ZION,48.15,7120999800,42


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [23]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            #'font_color': font_color,
            #'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            #'font_color': font_color,
            #'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            #'font_color': font_color,
            #'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [25]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [26]:
writer.close()